In [1]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import time
import codecs
import re
import json
import pandas as pd
import glob
import datetime
import random
import csv
from sklearn.preprocessing import MinMaxScaler
import random
from datetime import timedelta


In [2]:
startDate = '03-11-2021' 
endDate = '04-16-2021'

#get time period
startd = datetime.datetime.strptime(startDate, "%m-%d-%Y")
endd = datetime.datetime.strptime(endDate, "%m-%d-%Y")

#gets range of dates between start and end
date_array = (startd + datetime.timedelta(days=x) for x in range(0, (endd-startd).days))
dates = []
for date_object in date_array:
    #gets date value as string
    dates = np.append(dates, date_object.strftime("%Y-%m-%d"))

lastDate = "hi"

weights = []
with open('data/BodyMass.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)

    # for every row, print the row
    for row in csvReader:
        if row[7][:10] in dates and row[7][:10] != lastDate:
            lastDate = row[7][:10]
            weights.append(row[8])

print(weights)

['78.1499', '76.7501', '74.85', '75.95', '76.8', '76.1001', '75.3', '75.5998', '75.5', '74.2', '75.2002', '76.1999', '76.1001', '75.2501', '75.0001', '74.6999', '76.1001', '75.4501', '75.5998', '76', '76.35', '74.65', '74.5', '74', '74', '73.7001', '72.1502', '73.4502', '72.9998', '72.45', '72.1502', '73.4502', '73.8498', '73.2', '72.45', '72.5498']


In [3]:
with open('data/BodyFatPercentage.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    fats = []
    # for every row, print the row
    for row in csvReader:
        if row[7][:10] in dates:
            fats.append(row[8])
        

print(fats)

['0.09', '0.093', '0.09', '0.109', '0.096', '0.078', '0.084', '0.075', '0.081', '0.084', '0.084', '0.09', '0.087', '0.087', '0.09', '0.102', '0.102', '0.087', '0.087', '0.084', '0.081', '0.087', '0.087', '0.087', '0.087', '0.084', '0.084', '0.084', '0.081', '0.084', '0.084', '0.084', '0.078', '0.078', '0.078', '0.084']


In [4]:
with open('data/DietaryEnergyConsumed.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    calories = []
    calorie = 0
    lastDate = "2021-03-11"
    # for every row, print the row
    for row in csvReader:
        if row[7][:10] in dates:
            if row[7][:10] != lastDate:
                calories.append(calorie)
                calorie = float(row[8])
            else:
                calorie += float(row[8])
            lastDate = row[7][:10]
        
calories.append(calorie)
        

print(calories)

def caloriesBurned(weight, fat):
    fatMass = []
    leanMass = []
    for x in range(len(weight)):
        fatMass = np.append(fatMass, float(weight[x])*float(fat[x])/100)
        leanMass = np.append(leanMass, float(weight[x]) - fatMass[x])
    calories = [] 
    for x in range(len(weight) - 1):   
        calories = np.append(calories, -(fatMass[x]-fatMass[x+1])*442.39)
        if leanMass[x] >= leanMass[x+1]:
            calories[x] -= (leanMass[x] - leanMass[x+1])*70
        else:
            calories[x] -= (leanMass[x]-leanMass[x+1])*265
    calories = np.append(calories, [0])

    return calories
        
#calculate total calories burned in a day
calorieDiff = caloriesBurned(weights, fats)
print(calorieDiff)
for x in range(len(calories)):
    calories[x] = calories[x] - calorieDiff[x]

#calories = np.append(calories, [2000,2000, 2000])
print(calories)

[2172.2921, 2740.02, 2966.107, 2488.631, 2304.759, 2708.63, 2722.84, 1952.1239999999998, 3078.19, 2231.268, 3353.6, 3142.65, 2235.87, 2270.07, 3440.352, 2502.3900000000003, 3255.68, 2590.7200000000003, 3854.34, 2444.58, 3160.02, 2817.0, 3037.5, 2931.607, 1918.3, 2441.45, 1179.0, 3116.8, 1675.38, 1946.93, 2552.65, 1852.56, 3265.92, 1431.9, 2216.3, 1698.6929999999998]
[ -97.59771528 -134.50124951  294.2354425   223.64329007  -54.34421525
  -54.55694239   78.28471411   -5.32694669  -90.56318653  265.2020374
  265.88048891   -7.86961556  -59.7753824   -16.74311621  -17.77651284
  371.30634911  -49.96142398   39.6936031   105.71031339   92.39584086
 -117.84484622  -10.5485969   -35.16198965   -0.          -21.9135168
 -108.9778205   344.69370988  -32.48442041  -37.84244975  -21.07977972
  344.69370988  105.16753048  -45.67474364  -52.71784815   27.23298529
    0.        ]
[2269.8898152776296, 2874.5212495072706, 2671.871557504999, 2264.987709925002, 2359.10321525358, 2763.1869423864205, 264

In [5]:
rhr = []
rhpos = []
with open('data/RestingHeartRate.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    
    for row in csvReader:
        if row[7][:10] in dates:
            dt = row[7][:19]            
            dt = datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
            rhr.append(row[8])
            rhpos.append(dt)
            
            
print(np.size(dates))
print(np.size(rhr))

36
36


In [6]:
import math
#things to fill out
rootdir = "data"
startDate = '03-11-2021' 
endDate = '04-16-2021'

#get time period
startd = datetime.datetime.strptime(startDate, "%m-%d-%Y")
endd = datetime.datetime.strptime(endDate, "%m-%d-%Y")

#gets range of dates between start and end
date_array = (startd + datetime.timedelta(days=x) for x in range(0, (endd-startd).days))
dates = []
for date_object in date_array:
    #gets date value as string
    dates = np.append(dates, date_object.strftime("%Y-%m-%d"))

         


#get weight data
roughdata = []
hr = []
times = []
count = 0
lastDate = dates[0]
dts = []
lastRow = []
totaltimes = []
day = -1
lastDate = "hi"
with open('data/HeartRate.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)

    # for every row, print the row
    print(dates)
    for row in csvReader:
      #  print(row[7][:10])
    
        if row[7][:10] in dates:       
                
            dt = row[7][:19]            
            dtime = datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
            
            dts.append(dtime)
            hr.append(row[8])


            


    
splitInputs = []
lastSplit = 0
maxinterval = 30
done = False
counter = 0
day = 0
while done == False:
    counter += 1
    if counter < len(dts):
        if (dts[counter] - dts[counter - 1]).total_seconds() > maxinterval:
            dts.insert(counter, dts[counter - 1] + timedelta(seconds = maxinterval))
            date = dts[counter].strftime('%Y-%m-%d')
            
            
            #insert new indices with resting heart rate
            index = np.where(dates == date)
            hr.insert(counter, rhr[index[0][0]])
    else:
        done = True
    



totsecs = []
splitInputs = []
lastSplit = 0
for x in range(len(dts)-1):
    totsecs.append((dts[x+1] - dts[x]).total_seconds())
    if dts[x].strftime("%m/%d/%y") != dts[x+1].strftime("%m/%d/%y"):
        splitInputs.append(x- lastSplit)
        lastSplit = x


splitInputs.append(len(hr) - lastSplit)
totsecs.append(5)
    
hr = np.asarray(hr).astype(float)
print(len(hr))
print(len(dts))
print(np.sum(splitInputs))





['2021-03-11' '2021-03-12' '2021-03-13' '2021-03-14' '2021-03-15'
 '2021-03-16' '2021-03-17' '2021-03-18' '2021-03-19' '2021-03-20'
 '2021-03-21' '2021-03-22' '2021-03-23' '2021-03-24' '2021-03-25'
 '2021-03-26' '2021-03-27' '2021-03-28' '2021-03-29' '2021-03-30'
 '2021-03-31' '2021-04-01' '2021-04-02' '2021-04-03' '2021-04-04'
 '2021-04-05' '2021-04-06' '2021-04-07' '2021-04-08' '2021-04-09'
 '2021-04-10' '2021-04-11' '2021-04-12' '2021-04-13' '2021-04-14'
 '2021-04-15']
127540
127540
127540


In [7]:
startDate = '03-11-2021' 
endDate = '04-16-2021'
import math
#get time period
startd = datetime.datetime.strptime(startDate, "%m-%d-%Y")
endd = datetime.datetime.strptime(endDate, "%m-%d-%Y")
#gets range of dates between start and end
date_array = (startd + datetime.timedelta(days=x) for x in range(0, (endd-startd).days))
dates = []

for date_object in date_array:
    #gets date value as string
    dates = np.append(dates, date_object.strftime("%Y-%m-%d"))
    
roughdata = []
steps = []
times = []
count = 0
lastDate = dates[0]
stepdts = []
lastRow = []
totaltimes = []
date = []
day = -1
lastDate = "hi"
    
steps = []    
with open('data/DistanceWalkingRunning.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)

    # for every row, print the row
    count = 0
    for row in csvReader:
        if row[7][:10] in dates: 
            dt = row[7][:19]            
            dtime = datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
            stepdts.append(dtime)
            steps.append(row[8])

            
            
            
startstep = 0

while stepdts[startstep] <= dts[0]:
    startstep+=1 


    


    
hrsteps = np.zeros(len(hr))
print(hrsteps)


     
steppos = startstep
hrpos = 0
hrdate = dts[hrpos]
stepdate = stepdts[steppos]


while steppos != len(stepdts) - 1:    
    if steppos == 0:
        minstep = float(steps[steppos])
    else:
        
        if stepdts[steppos] == stepdts[steppos - 1]:
            minstep = float(steps[steppos]) / (stepdts[steppos] - stepdts[steppos - 2]).total_seconds()
        else:
            minstep = float(steps[steppos]) / (stepdts[steppos] - stepdts[steppos - 1]).total_seconds()


    if stepdts[steppos] < dts[hrpos+1]:
        hrsteps[hrpos] += minstep * (stepdts[steppos] - hrdate).total_seconds()
        hrdate = stepdts[steppos]
        steppos += 1
    else:
        hrsteps[hrpos] += minstep * (dts[hrpos+1] - hrdate).total_seconds()
        hrpos += 1
        if dts[hrpos + 1] == stepdts[steppos]:
            steppos += 1
              

print(hrsteps)



[0. 0. 0. ... 0. 0. 0.]
[1.382943 2.765886 4.148829 ... 0.       0.       0.      ]


In [8]:

hrinterval = np.multiply(hr,totsecs)
print(hrinterval)
'''for x in range(len(splitInputs)):
    if x == 0:
        number = np.sum(hrinterval[:splitInputs[x]])
        hrinterval[:splitInputs[x]] = (hrinterval[:splitInputs[x]] / number) * calories[x].detach().numpy()

    else:
        number = np.sum(hrinterval[np.sum(splitInputs[x-1]):np.sum(splitInputs[x-1] + splitInputs[x])]) 
        hrinterval[np.sum(splitInputs[x-1]):np.sum(splitInputs[x-1] + splitInputs[x])] = (hrinterval[np.sum(splitInputs[x-1]):np.sum(splitInputs[x-1] + splitInputs[x])] / number) * calories[x].detach().numpy()
'''                     
total = 0
for y in range(len(splitInputs)):
    summed = np.sum(hrinterval[total : total + splitInputs[y]])
    if y == 0:
        for x in range(splitInputs[y]):
            hrinterval[x] = hrinterval[x] / summed * calories[y]
    else:
        for x in range(splitInputs[y]):
            hrinterval[np.sum(splitInputs[:y]) + x] = hrinterval[np.sum(splitInputs[:y]) + x] / summed * calories[y]
        
        
    total += splitInputs[y]   
        

print(hrinterval)



[1770. 1560. 1560. ... 1470.  441.  335.]
[0.85061801 0.74969723 0.74969723 ... 0.55142728 0.16542818 0.1256654 ]


In [9]:
scaler = MinMaxScaler(feature_range=(-1,1))
print(hr)
normalized_hr = torch.FloatTensor(scaler.fit_transform(np.reshape(hr, (-1,1)))) 
normalized_interval =  torch.FloatTensor(scaler.fit_transform(np.reshape(totsecs, (-1,1))))
normalized_steps =  torch.FloatTensor(scaler.fit_transform(np.reshape(hrsteps, (-1,1))))

hr = np.hstack((normalized_hr, normalized_interval))
hr = np.hstack((hr, normalized_steps))
hr = torch.FloatTensor(hr)
print(hr)

[59. 52. 52. ... 49. 49. 67.]
tensor([[-0.8084,  1.0000, -0.9408],
        [-0.8922,  1.0000, -0.8815],
        [-0.8922,  1.0000, -0.8223],
        ...,
        [-0.9281,  1.0000, -1.0000],
        [-0.9281, -0.4000, -1.0000],
        [-0.7126, -0.6667, -1.0000]])


In [10]:
#calculate expected outputs
y_actual = []
day = 0


print(np.reshape(calories, (-1,1)))

outputScaler = MinMaxScaler(feature_range=(-1,1))
normalized_y = torch.FloatTensor(outputScaler.fit_transform(np.reshape(calories,(-1,1))))
print(np.shape(normalized_y))

outputs = normalized_y
print(outputs)
print(outputScaler.inverse_transform(outputs))

[[2269.88981528]
 [2874.52124951]
 [2671.8715575 ]
 [2264.98770993]
 [2359.10321525]
 [2763.18694239]
 [2644.55528589]
 [1957.45094669]
 [3168.75318653]
 [1966.0659626 ]
 [3087.71951109]
 [3150.51961556]
 [2295.6453824 ]
 [2286.81311621]
 [3458.12851284]
 [2131.08365089]
 [3305.64142398]
 [2551.0263969 ]
 [3748.62968661]
 [2352.18415914]
 [3277.86484622]
 [2827.5485969 ]
 [3072.66198965]
 [2931.607     ]
 [1940.2135168 ]
 [2550.4278205 ]
 [ 834.30629012]
 [3149.28442041]
 [1713.22244975]
 [1968.00977972]
 [2207.95629012]
 [1747.39246952]
 [3311.59474364]
 [1484.61784815]
 [2189.06701471]
 [1698.693     ]]
torch.Size([36, 1])
tensor([[-0.0148],
        [ 0.4001],
        [ 0.2611],
        [-0.0182],
        [ 0.0464],
        [ 0.3237],
        [ 0.2423],
        [-0.2292],
        [ 0.6021],
        [-0.2233],
        [ 0.5464],
        [ 0.5895],
        [ 0.0029],
        [-0.0032],
        [ 0.8006],
        [-0.1101],
        [ 0.6960],
        [ 0.1781],
        [ 1.0000],
      

In [11]:
class MyRNN(nn.Module):
    def __init__(self, input_size = 3, hidden_size = 5, output_size = 1):
        super(MyRNN, self).__init__()
        self.hidden_size = hidden_size
        self.in2hidden = nn.Linear(input_size + hidden_size, hidden_size)
        self.in2output = nn.Linear(input_size + hidden_size, output_size)
    
    def forward(self, x, hidden_state):
        combined = torch.cat((x, hidden_state), 1)
        hidden = torch.sigmoid(self.in2hidden(combined))
        output = self.in2output(combined)
        return output, hidden
    
    def init_hidden(self):
        return nn.init.kaiming_uniform_(torch.empty(1, self.hidden_size))

In [12]:
rnn = MyRNN()
print(rnn)

MyRNN(
  (in2hidden): Linear(in_features=8, out_features=5, bias=True)
  (in2output): Linear(in_features=8, out_features=1, bias=True)
)


In [13]:
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = torch.optim.Adam(rnn.parameters(), lr=0.5)   # optimize all cnn parameters
loss_fn = torch.nn.MSELoss(size_average=False)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma = 0.99)

C:\Users\peter\.conda\envs\ailab\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [ ]:

from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard
# default `log_dir` is "runs" - we'll be more specific here
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(log_dir)
print(outputs)
running_loss = 0
import random
counter = 0
for epoch in range(1000000):
    hidden_state = rnn.init_hidden() 
    x = random.randint(0,20)
    if x == 0:
        dailyInputs = hr[:int(splitInputs[x])]            
    else:           
        dailyInputs = hr[int(sum(splitInputs[:x])) :int(sum(splitInputs[:x])) + int(splitInputs[x])]
    for y in range(int(splitInputs[x])):            
        output, hidden_state = rnn(dailyInputs[y].view(1,3), hidden_state)                               # rnn output
       

    hidden = hidden_state
    loss = loss_fn(output, outputs[x].view(1, 1))                   #  loss
    optimizer.zero_grad()                           # clear gradients for this training step
    loss.backward()                                 # backpropagation, compute gradients
    optimizer.step() 
    scheduler.step()
        # apply gradients
    running_loss += loss.item()
    writer.add_scalar('running loss', running_loss, epoch)
    running_loss = 0

            
            
    print(scheduler.get_last_lr())
    
    print('Epoch: ', epoch, '| train loss: %.4f' % loss.data)

        
writer.close()

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
tensor([[-0.0148],
        [ 0.4001],
        [ 0.2611],
        [-0.0182],
        [ 0.0464],
        [ 0.3237],
        [ 0.2423],
        [-0.2292],
        [ 0.6021],
        [-0.2233],
        [ 0.5464],
        [ 0.5895],
        [ 0.0029],
        [-0.0032],
        [ 0.8006],
        [-0.1101],
        [ 0.6960],
        [ 0.1781],
        [ 1.0000],
        [ 0.0417],
        [ 0.6769],
        [ 0.3679],
        [ 0.5361],
        [ 0.4393],
        [-0.2411],
        [ 0.1777],
        [-1.0000],
        [ 0.5887],
        [-0.3968],
        [-0.2220],
        [-0.0573],
        [-0.3734],
        [ 0.7001],
        [-0.5537],
        [-0.0703],
        [-0.4068]])
[0.308645070471144]
Epoch:  0 | train loss: 0.4450
[0.308645070471144]
Epoch:  1 | train loss: 0.0383
[0.308645070471144]
Epoch:  2 | train loss: 0.0040
[0.308645070471144]
Epoch:  3 | train loss: 0.0702
[0.308645070471144]
E

[0.29947800323308055]
Epoch:  138 | train loss: 0.0442
[0.29947800323308055]
Epoch:  139 | train loss: 0.0705
[0.29947800323308055]
Epoch:  140 | train loss: 0.5286
[0.29947800323308055]
Epoch:  141 | train loss: 0.0379
[0.29947800323308055]
Epoch:  142 | train loss: 0.1328
[0.29947800323308055]
Epoch:  143 | train loss: 0.2552
[0.29947800323308055]
Epoch:  144 | train loss: 0.1526
[0.29947800323308055]
Epoch:  145 | train loss: 0.0000
[0.29947800323308055]
Epoch:  146 | train loss: 0.0471
[0.29947800323308055]
Epoch:  147 | train loss: 0.2965
[0.29947800323308055]
Epoch:  148 | train loss: 0.0034
[0.29947800323308055]
Epoch:  149 | train loss: 0.0580
[0.29947800323308055]
Epoch:  150 | train loss: 0.0117
[0.29947800323308055]
Epoch:  151 | train loss: 0.2184
[0.29947800323308055]
Epoch:  152 | train loss: 0.6289
[0.29947800323308055]
Epoch:  153 | train loss: 0.1021
[0.29947800323308055]
Epoch:  154 | train loss: 0.3869
[0.29947800323308055]
Epoch:  155 | train loss: 0.0000
[0.2994780

[0.29058320705905477]
Epoch:  289 | train loss: 0.0005
[0.29058320705905477]
Epoch:  290 | train loss: 0.0020
[0.29058320705905477]
Epoch:  291 | train loss: 0.5700
[0.29058320705905477]
Epoch:  292 | train loss: 0.3735
[0.29058320705905477]
Epoch:  293 | train loss: 0.0778
[0.29058320705905477]
Epoch:  294 | train loss: 0.0129
[0.29058320705905477]
Epoch:  295 | train loss: 0.6361
[0.29058320705905477]
Epoch:  296 | train loss: 0.4187
[0.29058320705905477]
Epoch:  297 | train loss: 0.0954
[0.29058320705905477]
Epoch:  298 | train loss: 0.1193
[0.29058320705905477]
Epoch:  299 | train loss: 0.0556
[0.29058320705905477]
Epoch:  300 | train loss: 0.8394
[0.29058320705905477]
Epoch:  301 | train loss: 0.0050
[0.29058320705905477]
Epoch:  302 | train loss: 0.0336
[0.29058320705905477]
Epoch:  303 | train loss: 0.8009
[0.29058320705905477]
Epoch:  304 | train loss: 0.0419
[0.29058320705905477]
Epoch:  305 | train loss: 0.4763
[0.29058320705905477]
Epoch:  306 | train loss: 0.1801
[0.2905832

[0.2819525952261938]
Epoch:  440 | train loss: 0.0366
[0.2819525952261938]
Epoch:  441 | train loss: 0.0188
[0.2819525952261938]
Epoch:  442 | train loss: 0.3530
[0.2819525952261938]
Epoch:  443 | train loss: 0.1723
[0.2819525952261938]
Epoch:  444 | train loss: 0.0272
[0.2819525952261938]
Epoch:  445 | train loss: 0.0076
[0.2819525952261938]
Epoch:  446 | train loss: 0.4152
[0.2819525952261938]
Epoch:  447 | train loss: 0.0068
[0.2819525952261938]
Epoch:  448 | train loss: 0.0003
[0.2819525952261938]
Epoch:  449 | train loss: 0.0098
[0.2819525952261938]
Epoch:  450 | train loss: 0.3824
[0.2819525952261938]
Epoch:  451 | train loss: 0.7849
[0.2819525952261938]
Epoch:  452 | train loss: 0.0637
[0.2819525952261938]
Epoch:  453 | train loss: 0.0548
[0.2819525952261938]
Epoch:  454 | train loss: 0.6258
[0.2819525952261938]
Epoch:  455 | train loss: 0.0719
[0.2819525952261938]
Epoch:  456 | train loss: 0.8286
[0.2819525952261938]
Epoch:  457 | train loss: 1.5133
[0.2819525952261938]
Epoch: 

[0.2735783211953806]
Epoch:  591 | train loss: 0.5648
[0.2735783211953806]
Epoch:  592 | train loss: 0.0119
[0.2735783211953806]
Epoch:  593 | train loss: 0.0105
[0.2735783211953806]
Epoch:  594 | train loss: 0.0547
[0.2735783211953806]
Epoch:  595 | train loss: 0.0862
[0.2735783211953806]
Epoch:  596 | train loss: 0.0007
[0.2735783211953806]
Epoch:  597 | train loss: 0.0824
[0.2735783211953806]
Epoch:  598 | train loss: 0.1153
[0.2735783211953806]
Epoch:  599 | train loss: 0.0040
[0.2735783211953806]
Epoch:  600 | train loss: 0.2450
[0.2735783211953806]
Epoch:  601 | train loss: 0.0266
[0.2735783211953806]
Epoch:  602 | train loss: 0.0042
[0.2735783211953806]
Epoch:  603 | train loss: 0.0005
[0.2735783211953806]
Epoch:  604 | train loss: 0.3662
[0.2735783211953806]
Epoch:  605 | train loss: 0.0266
[0.2735783211953806]
Epoch:  606 | train loss: 0.3213
[0.2735783211953806]
Epoch:  607 | train loss: 0.2600
[0.2735783211953806]
Epoch:  608 | train loss: 0.0512
[0.2735783211953806]
Epoch: 

...

[0.1606005372823957]
Epoch:  3261 | train loss: 0.7036
[0.1606005372823957]
Epoch:  3262 | train loss: 0.9905
[0.1606005372823957]
Epoch:  3263 | train loss: 0.2724
[0.1606005372823957]
Epoch:  3264 | train loss: 0.0959
[0.1606005372823957]
Epoch:  3265 | train loss: 0.0411
[0.1606005372823957]
Epoch:  3266 | train loss: 0.1723
[0.1606005372823957]
Epoch:  3267 | train loss: 0.2463
[0.1606005372823957]
Epoch:  3268 | train loss: 0.1700
[0.1606005372823957]
Epoch:  3269 | train loss: 0.1968
[0.1606005372823957]
Epoch:  3270 | train loss: 0.4573
[0.1606005372823957]
Epoch:  3271 | train loss: 0.1698
[0.1606005372823957]
Epoch:  3272 | train loss: 0.3014
[0.1606005372823957]
Epoch:  3273 | train loss: 0.0994
[0.1606005372823957]
Epoch:  3274 | train loss: 0.2047
[0.15899453190957175]
Epoch:  3275 | train loss: 0.4120
[0.15899453190957175]
Epoch:  3276 | train loss: 0.0015
[0.15899453190957175]
Epoch:  3277 | train loss: 0.0143
[0.15899453190957175]
Epoch:  3278 | train loss: 0.6536
[0.158

[0.15583054072457125]
Epoch:  3408 | train loss: 0.3180
[0.15583054072457125]
Epoch:  3409 | train loss: 0.0013
[0.15583054072457125]
Epoch:  3410 | train loss: 0.0053
[0.15583054072457125]
Epoch:  3411 | train loss: 0.2055
[0.15583054072457125]
Epoch:  3412 | train loss: 0.1281
[0.15583054072457125]
Epoch:  3413 | train loss: 0.0338
[0.15583054072457125]
Epoch:  3414 | train loss: 0.0596
[0.15583054072457125]
Epoch:  3415 | train loss: 0.1245
[0.15583054072457125]
Epoch:  3416 | train loss: 0.6369
[0.15583054072457125]
Epoch:  3417 | train loss: 0.0003
[0.15583054072457125]
Epoch:  3418 | train loss: 0.2255
[0.15583054072457125]
Epoch:  3419 | train loss: 0.0932
[0.15583054072457125]
Epoch:  3420 | train loss: 0.1173
[0.15583054072457125]
Epoch:  3421 | train loss: 0.2019
[0.15583054072457125]
Epoch:  3422 | train loss: 0.0250
[0.15583054072457125]
Epoch:  3423 | train loss: 0.4311
[0.15583054072457125]
Epoch:  3424 | train loss: 0.0061
[0.15427223531732553]
Epoch:  3425 | train loss:

[0.15120221783451074]
Epoch:  3555 | train loss: 0.4685
[0.15120221783451074]
Epoch:  3556 | train loss: 0.3817
[0.15120221783451074]
Epoch:  3557 | train loss: 0.0152
[0.15120221783451074]
Epoch:  3558 | train loss: 0.0444
[0.15120221783451074]
Epoch:  3559 | train loss: 0.0435
[0.15120221783451074]
Epoch:  3560 | train loss: 0.6476
[0.15120221783451074]
Epoch:  3561 | train loss: 0.6622
[0.15120221783451074]
Epoch:  3562 | train loss: 0.6054
[0.15120221783451074]
Epoch:  3563 | train loss: 0.0038
[0.15120221783451074]
Epoch:  3564 | train loss: 0.1172
[0.15120221783451074]
Epoch:  3565 | train loss: 0.0229
[0.15120221783451074]
Epoch:  3566 | train loss: 0.1431
[0.15120221783451074]
Epoch:  3567 | train loss: 0.1729
[0.15120221783451074]
Epoch:  3568 | train loss: 0.1808
[0.15120221783451074]
Epoch:  3569 | train loss: 0.0099
[0.15120221783451074]
Epoch:  3570 | train loss: 0.1782
[0.15120221783451074]
Epoch:  3571 | train loss: 0.0110
[0.15120221783451074]
Epoch:  3572 | train loss:

[0.14671136076260793]
Epoch:  3702 | train loss: 0.0014
[0.14671136076260793]
Epoch:  3703 | train loss: 0.2583
[0.14671136076260793]
Epoch:  3704 | train loss: 0.3594
[0.14671136076260793]
Epoch:  3705 | train loss: 0.1611
[0.14671136076260793]
Epoch:  3706 | train loss: 0.1348
[0.14671136076260793]
Epoch:  3707 | train loss: 0.8560
[0.14671136076260793]
Epoch:  3708 | train loss: 0.4218
[0.14671136076260793]
Epoch:  3709 | train loss: 0.2491
[0.14671136076260793]
Epoch:  3710 | train loss: 0.0631
[0.14671136076260793]
Epoch:  3711 | train loss: 0.0219
[0.14671136076260793]
Epoch:  3712 | train loss: 0.4373
[0.14671136076260793]
Epoch:  3713 | train loss: 0.1543
[0.14671136076260793]
Epoch:  3714 | train loss: 0.1032
[0.14671136076260793]
Epoch:  3715 | train loss: 0.1492
[0.14671136076260793]
Epoch:  3716 | train loss: 0.0192
[0.14671136076260793]
Epoch:  3717 | train loss: 0.0047
[0.14671136076260793]
Epoch:  3718 | train loss: 0.0002
[0.14671136076260793]
Epoch:  3719 | train loss:

[0.1423538866365977]
Epoch:  3849 | train loss: 0.0968
[0.1423538866365977]
Epoch:  3850 | train loss: 0.2412
[0.1423538866365977]
Epoch:  3851 | train loss: 0.0350
[0.1423538866365977]
Epoch:  3852 | train loss: 0.4015
[0.1423538866365977]
Epoch:  3853 | train loss: 0.1153
[0.1423538866365977]
Epoch:  3854 | train loss: 0.3118
[0.1423538866365977]
Epoch:  3855 | train loss: 0.0050
[0.1423538866365977]
Epoch:  3856 | train loss: 0.0017
[0.1423538866365977]
Epoch:  3857 | train loss: 0.1652
[0.1423538866365977]
Epoch:  3858 | train loss: 0.9006
[0.1423538866365977]
Epoch:  3859 | train loss: 0.0037
[0.1423538866365977]
Epoch:  3860 | train loss: 0.6622
[0.1423538866365977]
Epoch:  3861 | train loss: 0.0714
[0.1423538866365977]
Epoch:  3862 | train loss: 0.2010
[0.1423538866365977]
Epoch:  3863 | train loss: 0.2870
[0.1423538866365977]
Epoch:  3864 | train loss: 0.0111
[0.1423538866365977]
Epoch:  3865 | train loss: 0.1106
[0.1423538866365977]
Epoch:  3866 | train loss: 0.0611
[0.1423538

In [20]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 9316), started 1:11:20 ago. (Use '!kill 9316' to kill it.)

In [35]:
for x in range(len(splitInputs)):
    outputlist = []

    output = 0
    
    if x == 0:
        for y in range(len(hr[:splitInputs[x]])):
            outputlist.append(rnn(hr[y]).detach().numpy()[0])
    else:
        for y in range(len(hr[np.sum(splitInputs[x-1]):np.sum(splitInputs[x-1] + splitInputs[x])])):
            outputlist.append(rnn(hr[y]).detach().numpy()[0]   )                

    outputlist = np.reshape(outputlist, (-1,1))
    outputlist = outputScaler.inverse_transform(outputlist)
    output = torch.FloatTensor([np.sum(outputlist)])
    
    calories = torch.FloatTensor(calories)
    loss = loss_fn(output.view(1,1), calories[x].view(1,1))

    print(output, " : ", calories[x], 'train loss: %.4f' % loss.data)
    
    
outputList = np.reshape(outputList, (-1,1))


tensor([2533.1365])  :  tensor(2269.8899) train loss: 69298.7656
tensor([2321.6406])  :  tensor(2874.5212) train loss: 305676.9688
tensor([3723.0654])  :  tensor(2671.8716) train loss: 1105008.5000
tensor([1972.4058])  :  tensor(2264.9878) train loss: 85604.2422
tensor([2264.6121])  :  tensor(2359.1033) train loss: 8928.5889
tensor([2140.5583])  :  tensor(2763.1870) train loss: 387666.4375
tensor([2910.4272])  :  tensor(2644.5552) train loss: 70687.9609
tensor([2600.2766])  :  tensor(1957.4509) train loss: 413224.8750
tensor([2208.3772])  :  tensor(3168.7532) train loss: 922322.0000
tensor([2136.3784])  :  tensor(1966.0659) train loss: 29006.3477
tensor([2596.9326])  :  tensor(3087.7195) train loss: 240871.7500
tensor([2368.5649])  :  tensor(3150.5195) train loss: 611453.0000
tensor([2420.1099])  :  tensor(2295.6453) train loss: 15491.4365
tensor([2294.9260])  :  tensor(2286.8132) train loss: 65.8174
tensor([2610.3086])  :  tensor(3458.1284) train loss: 718798.4375
tensor([1973.9701]) 

NameError: name 'outputList' is not defined

In [ ]:
outputList = []
for x in range(36):
    hidden_state = rnn.init_hidden()

    if x == 0:
        dailyInputs = hr[:int(splitInputs[x])]            
    else:           
        dailyInputs = hr[int(sum(splitInputs[:x])) :int(sum(splitInputs[:x])) + int(splitInputs[x])]
    
    for y in range(int(splitInputs[x])):     
        output, hidden_state = rnn(dailyInputs[y].view(1,2), hidden_state)
   
    outputList.append(output.detach().numpy())
    print(output)
    print(outputs[x].view(1,1))
    loss = loss_fn(output, outputs[x].view(1, 1))  
    print(output[0][0], " : ", outputs[x][0], 'train loss: %.4f' % loss.data)
    
outputList = np.reshape(outputList, (-1,1))
print(outputScaler.inverse_transform(outputList))

    

